# PyCity Schools Analysis

## Trend Summary
* The top 5 preforming schools did not appear to be influenced by the spending per student or the class size. They ranged from 578-638 spent per student and school sizes of 962-2283. 

* That being said, the bottom 5 preforming schools did have larger than average budgets per student (ranging from 637-665) and larger student bodies (2917-4761).

* The average passing rate for both reading and math per school largely didn't change more than a percentage point per grade. If a school was making a difference we would expect to see these numbers trending upwards each year but since they are not this seems to suggest that the passing rates of the schools have little to do with anything other than the caliber of students that are admitted to the school.

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (645 -675) underperformed compared to schools with smaller budgets (585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 


In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import os

# File to Load (Remember to Change These)
school_data_to_load = os.path.join ("Resources","schools_complete.csv")
student_data_to_load = os.path.join ("Resources","students_complete.csv")

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
# Add a column to calculate the total average score
school_data_complete ["Tot Score"] = (school_data_complete["math_score"] +school_data_complete["reading_score"])/2

#Calculate the requested variables
totschool = school_data_complete["school_name"].nunique()
totstudent = school_data_complete["Student ID"].nunique()
totbudget = school_data["budget"].sum()
avgmath = school_data_complete["math_score"].mean()
avgread = school_data_complete["reading_score"].mean()
avgtot = school_data_complete["Tot Score"].mean()
mathpass = (school_data_complete["math_score"]>= 70).sum ()/totstudent * 100
readpass = (school_data_complete["reading_score"]>= 70).sum ()/totstudent * 100
#totalpass = (school_data_complete["Tot Score"]>= 70).sum ()/totstudent * 100
totalpass = (mathpass + readpass) /2
# Create a dataframe to hold the above results
school_summary_df = pd.DataFrame({"Total Schools": [totschool],
                                    "Total Students": [totstudent],
                                    "Total Budget": [totbudget],
                                    "Average Math Score": [avgmath],
                                    "Average Reading Score": [avgread],
#                                    "Overall Passing Score": [avgtot],
                                    "% Passing Math": [mathpass],
                                    "% Passing Reading":[readpass],
                                    "% Overall Passing Rate": [totalpass]
                                   })
# Optional: give the displayed data cleaner formatting
school_summary_df["Total Students"] = school_summary_df["Total Students"].map("{:,}".format)
school_summary_df["Total Budget"] = school_summary_df["Total Budget"].map("${:,.2f}".format)


school_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,80.393158


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [3]:
#create a data frame with the total passing scores
school_pass_df = school_data_complete
school_pass_df["math pass"]= school_pass_df["math_score"]>=70
school_pass_df["read pass"]= school_pass_df["reading_score"]>=70
school_pass_df["tot pass"]= school_pass_df["Tot Score"]>=70

#Create a grouped dataframe
school_group_df = school_data_complete.groupby(["school_name"])
school_df = school_data.groupby(["school_name"])
school_group_pass_df = school_pass_df.groupby(["school_name"])

#Create the summary table and add elements from the grouped dataframe
school_summary = school_data_complete.groupby(["school_name"]).first()[["type"]]
school_summary = school_summary.rename(columns={"type":"School Type"})

school_summary["Total Students"] = school_group_df.count()[["Student ID"]]
school_summary["Total School Budget"] = school_df.sum()[["budget"]]
school_summary["Per Student Budget"] = school_summary["Total School Budget"]/school_group_df["Student ID"].nunique()
school_summary["Average Math Score"]= school_group_df.mean()[["math_score"]]
school_summary["Average Reading Score"]= school_group_df.mean()[["reading_score"]]
school_summary["Average Total Score"]=school_group_df.mean()[["Tot Score"]]
school_summary["Passing Math"]=school_group_pass_df.sum()[["math pass"]]
school_summary["% Passing Math"]= school_summary["Passing Math"]/school_summary["Total Students"]*100
school_summary["Passing Reading"]=school_group_pass_df.sum()[["read pass"]]
school_summary["% Passing Reading"]= school_summary["Passing Reading"]/school_summary["Total Students"]*100
#school_summary["Passing Overall"]=school_group_pass_df.sum()[["tot pass"]]
#school_summary["% Passing Overall"]= school_summary["Passing Overall"]/school_summary["Total Students"]*100
school_summary["% Passing Overall"]= (school_summary["% Passing Reading"]+ school_summary["% Passing Math"])/2

# Clean up the dataset, format
del school_summary["Average Total Score"]
del school_summary ["Passing Reading"]
del school_summary ["Passing Math"]
#del school_summary ["Passing Overall"]

school_summary["Total School Budget"]=school_summary["Total School Budget"].map("{:,.2f}".format)
school_summary["Per Student Budget"]=school_summary["Per Student Budget"].map("{:,.2f}".format)

school_summary.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
school_name,,,,,,,,,
Bailey High School,District,4976,"3,124,928.00",628.00,77.048432,81.033963,66.680064,81.933280,74.306672
Cabrera High School,Charter,1858,"1,081,356.00",582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Figueroa High School,District,2949,"1,884,411.00",639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Ford High School,District,2739,"1,763,916.00",644.00,77.102592,80.746258,68.309602,79.299014,73.804308
Griffin High School,Charter,1468,"917,500.00",625.00,83.351499,83.816757,93.392371,97.138965,95.265668


## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [4]:
# Sort and display the top five schools in overall passing rate
school_summary = school_summary.sort_values ("% Passing Overall", ascending=False)
school_summary.head(5)


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
school_name,,,,,,,,,
Cabrera High School,Charter,1858,"1,081,356.00",582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,Charter,1635,"1,043,130.00",638.00,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,Charter,962,"585,858.00",609.00,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,Charter,1468,"917,500.00",625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,2283,"1,319,574.00",578.00,83.274201,83.989488,93.867718,96.539641,95.203679


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [5]:
school_summary = school_summary.sort_values ("% Passing Overall", ascending=True)
school_summary.head(5)


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
school_name,,,,,,,,,
Rodriguez High School,District,3999,"2,547,363.00",637.00,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,2949,"1,884,411.00",639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,2917,"1,910,635.00",655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,"3,094,650.00",650.00,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,2739,"1,763,916.00",644.00,77.102592,80.746258,68.309602,79.299014,73.804308


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [6]:
# Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

# * Create a pandas series for each grade. Hint: use a conditional statement.
# * Group each series by school
# * Combine the series into a dataframe
math_summary = school_data_complete.loc [school_data_complete["grade"]>= "9th"]
math_summary = math_summary.groupby ("school_name")
math_summary = math_summary.mean()[["math_score"]]
math_summary = math_summary.rename(columns={"math_score":"9th"})

ten = school_data_complete.loc [school_data_complete["grade"]>= "10th"]
ten = ten.groupby ("school_name")
math_summary ["10th"] = ten["math_score"].mean()
          
eleven = school_data_complete.loc [school_data_complete["grade"]>= "11th"]
eleven = eleven.groupby ("school_name")
math_summary ["11th"]= eleven.mean()[["math_score"]]

twelve = school_data_complete.loc [school_data_complete["grade"]>= "12th"]
twelve = twelve.groupby ("school_name")
math_summary["12th"] = twelve.mean()[["math_score"]]

# Optional: give the displayed data cleaner formatting
math_summary

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,77.048432,77.065561,76.839099
Cabrera High School,83.094697,83.061895,83.030891,83.171429
Figueroa High School,76.403037,76.711767,76.771729,76.717671
Ford High School,77.361345,77.102592,76.903988,76.897230
Griffin High School,82.044010,83.351499,83.016008,82.590585
Hernandez High School,77.438495,77.289752,77.272594,77.336638
Holden High School,83.787402,83.803279,83.939297,83.419048
Huang High School,77.027251,76.629414,76.886512,77.108467
Johnson High School,77.187857,77.072464,77.204867,77.057791


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [7]:
read_summary = school_data_complete.loc [school_data_complete["grade"]>= "9th"]
read_summary = read_summary.groupby ("school_name")
read_summary = read_summary.mean()[["reading_score"]]
read_summary = read_summary.rename(columns={"reading_score":"9th"})

ten = school_data_complete.loc [school_data_complete["grade"]>= "10th"]
ten = ten.groupby ("school_name")
read_summary ["10th"] = ten["reading_score"].mean()
          
eleven = school_data_complete.loc [school_data_complete["grade"]>= "11th"]
eleven = eleven.groupby ("school_name")
read_summary ["11th"]= eleven.mean()[["reading_score"]]

twelve = school_data_complete.loc [school_data_complete["grade"]>= "12th"]
twelve = twelve.groupby ("school_name")
read_summary["12th"] = twelve.mean()[["reading_score"]]

read_summary

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,81.033963,81.075997,81.141593
Cabrera High School,83.676136,83.975780,83.882902,83.932967
Figueroa High School,81.198598,81.158020,81.070448,81.276913
Ford High School,80.632653,80.746258,80.566224,80.644315
Griffin High School,83.369193,83.816757,83.858757,83.637660
Hernandez High School,80.866860,80.934412,81.033157,80.862931
Holden High School,83.677165,83.814988,83.993610,84.080952
Huang High School,81.290284,81.182722,81.065116,80.887334
Johnson High School,81.260714,80.966394,81.033390,81.247432


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [8]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

school_summary[['Per Student Budget']] = school_summary[['Per Student Budget']].replace('[\$,]','',regex=True).astype(float)
school_summary["Spending Ranges (Per Student)"] = pd.cut(school_summary["Per Student Budget"], spending_bins, labels=group_names) 
#You can then get data by that binned column
spend_df = school_summary.groupby("Spending Ranges (Per Student)")

spend_summary = spend_df.mean()[["Average Math Score"]]
spend_summary["Average Reading Score"] = spend_df.mean()[["Average Reading Score"]]
spend_summary["% Passing Math"] = spend_df.mean()[["% Passing Math"]]
spend_summary["% Passing Reading"] = spend_df.mean()[["% Passing Reading"]]
spend_summary["% Overall Passing Rate"] = (spend_summary["% Passing Math"]+spend_summary["% Passing Reading"])/2

spend_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,95.035486
$585-615,83.599686,83.885211,94.230858,95.900287,95.065572
$615-645,79.079225,81.891436,75.668212,86.106569,80.887391
$645-675,76.997210,81.027843,66.164813,81.133951,73.649382


## Scores by School Size

* Perform the same operations as above, based on school size.

In [9]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

school_summary[['Total Students']] = school_summary[['Total Students']].astype(float)
school_summary["School Size"] = pd.cut(school_summary["Total Students"], size_bins, labels=group_names) 
#You can then get data by that binned column
size_df = school_summary.groupby("School Size")

size_summary = size_df.mean()[["Average Math Score"]]
size_summary["Average Reading Score"] = size_df.mean()[["Average Reading Score"]]
size_summary["% Passing Math"] = size_df.mean()[["% Passing Math"]]
size_summary["% Passing Reading"] = size_df.mean()[["% Passing Reading"]]
size_summary["% Overall Passing Rate"] = (size_summary["% Passing Math"]+size_summary["% Passing Reading"])/2

size_summary


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


## Scores by School Type

* Perform the same operations as above, based on school type.

In [10]:
type_df = school_summary.groupby("School Type")

type_summary = type_df.mean()[["Average Math Score"]]
type_summary["Average Reading Score"] = type_df.mean()[["Average Reading Score"]]
type_summary["% Passing Math"] = type_df.mean()[["% Passing Math"]]
type_summary["% Passing Reading"] = type_df.mean()[["% Passing Reading"]]
type_summary["% Overall Passing Rate"] = (type_summary["% Passing Math"]+type_summary["% Passing Reading"])/2

type_summary


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757


### For a summary of the trends please see the top of the document